In [1]:
# Importēt nepieciešamās bibliotēkas

import gzip
# from rdflib.namespace import RDF
# from rdflib import Dataset, Namespace, RDF, RDFS, URIRef, Literal, BNode
from collections import Counter

In [2]:
# Funkcija, kas izveido RDF četrinieku sarakstu no .gz faila
# Jāpadod faila vārds un Schema.org klases vārds, piemēram, School

def parse_gz_file(filename, schema_org_class_name):
    current_subject_has_class = False
    quads = []
    
    with gzip.open(filename, 'rt', encoding='utf-8') as f:
        for line in f:

            line = line.strip()
            if not line or line.startswith('#'):
                continue
            
            try:
                s, p, o, g, _ = line.split()
            except ValueError:
                continue

            # Datu kopā RDF četriniekiem ir šāda struktūra:
            # Sākumā nāk četrinieks, kas norāda aprakstamās entītijas tipu
            # Tam seko pārējie četrinieki, kas apraksta entītijas īpašības
            # Tātad, datu apstrāde notiek blokos: sākumā nolasām tipu, tad nolasām īpašības

            # Datu kopa satur ne tikai entītijas ar tipu X, bet arī citas
            # Mēs gribam analizēt tikai entītijas ar tipu X
            # Lai noskaidrotu tipu, mēs nolasām predikātu, kas apzīmē tipu, un pārbaudām, vai tā vērtība atbilst schema.org tipam X
            # Ja tips nav X, tad datu bloks tiek ignorēts

            if p == "<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>":
                if o == f"<http://schema.org/{schema_org_class_name}>":
                    current_subject_has_class = True
                else:
                    current_subject_has_class = False

            if current_subject_has_class:
                quads.append((s, p, o, g))
    
    return quads


In [ ]:
# Iegūt četriniekus klasei School 2024., 2022. un 2019. gadu datu kopās

school_quads = {}
school_quads["2024"] = parse_gz_file("../school_2024.gz", "School")
school_quads["2022"] = parse_gz_file("../school_2022.gz", "School")
school_quads["2019"] = parse_gz_file("../school_2019.gz", "School")

In [ ]:


for year, quads in school_quads.items():
    # Top 10 predikāti (predicates) entītijām ar tipu School
    
    predicate_counter = Counter()

    for s, p, o, graph in quads:
        predicate_counter[p] += 1

    top_predicates = predicate_counter.most_common(10)

    print(f"\nTop 10 predikāti {year}:")

    # Īpašība; entītiju skaits, kas izmanto; procents no kopējā entītiju skaita
    # Tā kā katra entītija sākas ar tipa predikātu, tipa predikātu skaitu var uzskatīt par entītiju skaitu
    for p, count in top_predicates:
        print(p, count, str(round(100 * (count / (predicate_counter["<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>"])), 2)) + "%")


Top 10 predikāti 2024:
<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> 291497 100.0%
<http://schema.org/sameAs> 180817 62.03%
<http://schema.org/url> 162041 55.59%
<http://schema.org/address> 130212 44.67%
<http://schema.org/logo> 89811 30.81%
<http://schema.org/contactPoint> 70733 24.27%
<http://schema.org/image> 58995 20.24%
<http://schema.org/location> 54206 18.6%
<http://schema.org/telephone> 42753 14.67%
<http://schema.org/name> 35439 12.16%

Top 10 predikāti 2022:
<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> 395869 100.0%
<http://schema.org/sameAs> 184186 46.53%
<http://schema.org/url> 172132 43.48%
<http://schema.org/address> 167676 42.36%
<http://schema.org/image> 103779 26.22%
<http://schema.org/logo> 86100 21.75%
<http://schema.org/name> 60668 15.33%
<http://schema.org/contactPoint> 58429 14.76%
<http://schema.org/location> 48651 12.29%
<http://schema.org/review> 43808 11.07%

Top 10 predikāti 2019:
<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> 128666 100.0%
<http